### Visualización Pasiva
Los puntos 1 y 2 deben ser entregados en formato ipynb.

1) Tomando los datos de Covid-19 en Argentina transformados en el TP3, analizar y justificar el siguiente desarrollo en función del Modelo Unificado de Visualización:
- Graficar las curvas de crecimiento de casos totales para las 10 provincias con mayor cantidad de casos.
- Calcular la penetración de casos (casos cada 100 mil hab.) por provincia y grafique la curva de crecimiento para las 10 provincias con mayor penetración. Comparar los gráficos con el punto anterior.
- Analizar varias relaciones entre dos variables buscando información valiosa. Por ejemplo, cantidad de casos vs  población total, o vs densidad demográfica, etc.
- Comparar gráficos y sacar conclusiones.

2) Graficar en un mapa del país, un círculo por cada provincia que cuenta con casos de Covid-19 registrados.
- Indicar la cantidad de casos con un círculo de diferente tamaño.
- Indicar la penetración de casos por provincia utilizando una gama de colores para cada círculo.

In [ ]:
def get_color(cant_casos, casos_provincia):
    """
    Devuelve un codigo hexadecimal del color dentro de la gama rojo-amarillo
    """
    tope_gama = 150
    salto = round(tope_gama/len(cant_casos))
    indice = cant_casos.index(casos_provincia)
    return '#%02x%02x%02x' % (254, tope_gama - (salto*indice), 49)

In [ ]:
def cambiar_nombre(nombre):
    """
    Unifica los nombres de las provincias en los df para poder mergear
    """
    if ("Provincia de" in nombre):
        return " ".join(nombre.split(" ")[2:])
    else: #Ciudad autonoma de Buenos aires
        return "CABA"

In [ ]:
def top_10_casos_confirmados(df):
    """
    Devuelve la lista de 10 provincias con mas casos confirmados
    """
    #Agrupo el df por provincia y me quedo con los valores mas altos de casos
    idx = df.groupby(['provincia'])['tot_casosconf'].transform(max) == df['tot_casosconf']
    df = df[idx]
    return df.sort_values(by='tot_casosconf', ascending=True).provincia.to_list()[:10]

In [ ]:
def top_10_penetracion(df):
    """
    Devuelve la lista de 10 provincias con mas casos por penetracion de habitantes
    """
    #Agrupo el df por provincia y me quedo con los valores mas altos de casos
    idx = df.groupby(['provincia'])['penetracion_casos'].transform(max) == df['penetracion_casos']
    df = df[idx]
    return df.sort_values(by='penetracion_casos', ascending=True).provincia.to_list()[:10]

In [ ]:
def calcular_penetracion(tot_poblacion, tot_casos):
    return round(tot_casos/1000)

In [ ]:
def armar_grafico(df, agrupamiento):
    """
    Muestra los graficos por provincia
    """
    df = df.drop_duplicates(['fecha','provincia'])
    r=df.pivot(index='fecha',columns='provincia', values=[agrupamiento])
    r.columns = r.columns.droplevel(0)
    r = r.reset_index().rename_axis(None, axis=1)
    r = r.set_index(['fecha'])
    r.index = pd.to_datetime(r.index)
    r.iloc[0] = 0

    #Armamos una figura
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=r.index,y=r[r.columns[0]], visible=True))
    updatemenu = []
    buttons = []

    # Creamos un boton para cada provincia en el dataframe
    for col in r.columns:
        buttons.append(dict(method='restyle',
                            label=col,
                            visible=True,
                            args=[{'y':[r[col]],
                                   'x':[r.index],
                                   'type':'scatter'}, [0]],
                            )
                      )

    #Ultimos retoques de visualizacion
    updatemenu = []
    menu = dict()
    updatemenu.append(menu)

    updatemenu[0]['buttons'] = buttons
    updatemenu[0]['direction'] = 'down'
    updatemenu[0]['showactive'] = True

    #Agregagos los botones filtro a la figura
    fig.update_layout(showlegend=False, updatemenus=updatemenu)
    return fig

In [ ]:
import pandas as pd
import numpy as np
#Mapas por provincia
import plotly.graph_objects as go
from datetime import datetime

#leemos los datos
df_covid =pd.read_csv('export')
df_covid2= pd.read_json('pop_provs.json')

#Reemplazar los nombres que dicen "Provincia de .." dejando solo el nombre de la provincia. Para poder mergear
df_covid2.name = df_covid2.name.apply(lambda row: cambiar_nombre(row))

#luego de preparadas las columnas de nombres de provincia en ambos dataframes, podemos joinear.
df_covid.fecha = df_covid.fecha.apply(lambda row: datetime.strptime(row, '%d/%m/%Y'))

#Renombramos algunas columnas
df_covid.rename(columns={'osm_admin_level_4':'provincia'}, inplace=True)
df_covid2.rename(columns={'name':'provincia'}, inplace=True)

#Borro la fila de provincia que dice "Indeterminado"
df_covid = df_covid[df_covid.provincia != 'Indeterminado']

#Listo los df para mergear
resultado = df_covid.merge(df_covid2, left_on='provincia', right_on='provincia')

#Calulamos la penetracion de casos para cada provincia en cada fecha
resultado['penetracion_casos'] = resultado.apply(lambda row: calcular_penetracion(row.total_pop, row.tot_casosconf), axis=1)

#Obtengo las 10 provincias con mas casos confirmados
top_10_casos = top_10_casos_confirmados(resultado)

#Obtengo las 10 provincias con mayor penetracion de casos
top_10_mayor_penetracion = top_10_penetracion(resultado)

#Obtenemos solo el top10 con mas casos
resultado1 = resultado.loc[resultado.provincia.isin(top_10_casos)]

#Obtenemos solo el top10 con mayor penetracion
resultado2 = resultado.loc[resultado.provincia.isin(top_10_mayor_penetracion)]

### TOP 10 Provincias con mas casos confirmados

In [ ]:
figura1 = armar_grafico(resultado1,'tot_casosconf')
figura1

### TOP 10 Provincias con mayor penetracion de casos

In [ ]:
figura2 = armar_grafico(resultado2,'penetracion_casos')
figura2

Del resultado de los graficos podemos observar que el crecimiento de las curvas es bastante similar en ambos graficos(total de casos confirmados y de casos por cada 1000 mil habitantes)
Ademas se puede ver que las provincias con mas penetracion de casos cada 1000 mil habitantes tienen casi la misma cantidad de penetracion por persona a pesar de tener distinas densidades demograficas.

### Visualizacion pasiva

In [ ]:
def crear_circulos(df_datos, need_color=False):
    """
    Crea los circulos similares a puntos de interes dentro del mapa con radio
    acorde a la cantidad de casos confirmados.
    """
    r = []
    constante = 1
    lista_casos_confirmados = sorted(df_datos.tot_casosconf.to_list())
    df_datos=df_datos[['coordenadas','tot_casosconf']]
    
    for d in df_datos.itertuples():
        coordenadas = [float(d.coordenadas.split(',')[0]), float(d.coordenadas.split(',')[1])]
        radio = d.tot_casosconf * constante
        circulo = folium.Circle(location = coordenadas,
                            radius = radio,
                            color = get_color(lista_casos_confirmados,d.tot_casosconf) if need_color else '#3186cc',
                            fill = True,
                            fill_color =get_color(lista_casos_confirmados,d.tot_casosconf) if need_color else '#3186cc').add_child(folium.Popup('Casos Confirmados: '+ str(d.tot_casosconf)))
        r.append(circulo)
    return r

In [ ]:
def crear_mapa_argentina(df,need_color=False):
    """
    Crea el mapa de argentina con circulos por provincia del tamaño acorde
    a la cantidad de casos confirmados por provincia
    """
    points = [(-34.6083, -58.3712)] #Argentina-BuenosAires
    style2 = {'fillColor': '#8c96c6', 'color': '#787878', 'weight': 1.5, 'fillOpacity': 0.7}
    county_geojson = json.load(open('provincia.json'))
    
    m1 = folium.Map(location=points[0], zoom_start=5)
    folium.GeoJson(county_geojson,style_function=lambda x:style2).add_to(m1)
    lista_circulos = crear_circulos(datos, need_color)
    for circulo in lista_circulos:
        circulo.add_to(m1)
    return m1

In [ ]:
import json
import folium
import numpy as np
from geopy.distance import geodesic

#Agrupo el df por provincia y me quedo con los valores mas altos de casos
idx = resultado.groupby(['provincia'])['tot_casosconf'].transform(max) == resultado['tot_casosconf']
df = resultado[idx]

#Abro el csv de coordenadas
df_coordenadas = pd.read_csv('coordenadas.csv', delimiter=';')

#Merge
datos =df.merge(df_coordenadas, left_on='provincia', right_on='provincia')

In [ ]:
#Obtenemos graficos
crear_mapa_argentina(datos)

In [ ]:
#Obtenemos graficos
crear_mapa_argentina(datos, True)